# API测试

In [19]:
import hashlib
import random
import math
import requests

BASE_URL = 'https://www.newrank.cn'
GET_ACCOUNT_ARTICLES_URL = '/xdnphb/detail/getAccountArticle'
SEARCH_OFFICIAL_WEIXIN_ACCOUNT = '/xdnphb/data/weixinuser/searchWeixinDataByCondition'

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.84 Safari/537.36'
token = 'AA79F89BA13E514FE5BB69C90B7E8578'
headers = {
    'User-Agent': user_agent,
    'Cookie': 'token=%s; tt_token=true; ' % token,
    'x-requested-with': 'XMLHttpRequest',
    'content-type':'application/x-www-form-urlencoded; charset=UTF-8'
}

In [20]:
def generate_nonce():
    chars = '0123456789abcdef'
    res = ''
    for _ in range(9):
        res += chars[math.floor(16 * random.random())]
    return res


def xyz(params_list, api_suffix):
    md5_str = api_suffix + '?AppKey=joker&' + '&'.join(map(lambda x: '='.join(x), params_list))
    return hashlib.md5(md5_str.encode()).hexdigest()


def form_encoding(params, api_suffix, nonce=None):
    params_list = [entry for entry in params.items()]
    params_list.sort(key=lambda x: x[0])
    params_list.append(('nonce', nonce if nonce else generate_nonce()))

    xyz_param = xyz(params_list, api_suffix)
    params_list.append(('xyz', xyz_param))
    return '&'.join(map(lambda x: '='.join(x), params_list))


def get_account_articles(uuid):
    params = {
        'uuid': uuid,
        'flag':'true'
    }

    data = form_encoding(params, GET_ACCOUNT_ARTICLES_URL)
    print(data)
    return requests.post(BASE_URL + GET_ACCOUNT_ARTICLES_URL, data=data, headers=headers)

def search_weixin_official_account(keyword, filter='', order=''):
    params = {
        'keyName': keyword,
        'order': order,
        'filter': filter,
        'hasDeal': 'false',
    }
    data = form_encoding(params, SEARCH_OFFICIAL_WEIXIN_ACCOUNT)
    print(data)
    return requests.post(BASE_URL + SEARCH_OFFICIAL_WEIXIN_ACCOUNT, data=data, headers=headers)


In [22]:
resp = get_account_articles('A4C71D652EB75E74E08AF6D7687B2162')
resp

flag=true&uuid=A4C71D652EB75E74E08AF6D7687B2162&nonce=7977f33fc&xyz=c14d75fb8d1977ef5b3187949351a50b


<Response [200]>

In [23]:
len(resp.json())

2

In [40]:
sear_res = search_weixin_official_account('Me')
sear_res

filter=&hasDeal=false&keyName=Me&order=&nonce=6f9d45ee8&xyz=285b75df7a644b890398cbf9836c10e3


<Response [200]>

In [59]:
obj = sear_res.json()['value']['result'][0]
obj['uuid'], obj['description']

('D53E8970386901A0A565040790E5C40C',
 "新浪微博@韩国<font color='red'>me</font>2day专属帐号，合作联系：<font color='red'>me</font>2day@qq.com")

# 搜索公众号参数
* order: relation(综合-默认), NRI(新榜指数), collect(收藏人数), 
* keyName: 关键字
* filter(范围): nickname, tags, auth(认证主体), top500, ori(原创), server(服务号), certified(认证号)

# 面对对象测试

In [7]:
import hashlib
import logging
import random
import math
import functools
import requests

from urllib.parse import urlencode

BASE_URL = 'https://www.newrank.cn'
GET_ACCOUNT_ARTICLES_URL = '/xdnphb/detail/getAccountArticle'
SEARCH_OFFICIAL_WEIXIN_ACCOUNT = '/xdnphb/data/weixinuser/searchWeixinDataByCondition'
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.84 Safari/537.36'
token = 'AA79F89BA13E514FE5BB69C90B7E8578'


def check_response_return_json(func, ret=None):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        resp = func(*args, **kwargs)
        if resp.status_code != 200 or resp.text == '':
            logging.error(
                f' [{func.__name__} error]: code {resp.status_code}, text: {resp.text}, args: {args}, kwargs: {kwargs}')
            retu = [] if ret is None else ret
            return retu
        else:
            return resp.json()

    return func


class NewrankClient():
    def __init__(self, token):
        self.token = token
        self.headers = {
            'User-Agent': user_agent,
            'Cookie': 'token=%s; tt_token=true; ' % token,
            'X-Requested-With': 'XMLHttpRequest',
            'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
            'Origin': 'https://www.newrank.cn',
            'Accept': 'application/json, text/javascript, */*; q=0.01',
            'Accept-Encoding': 'gzip, deflate, br',

        }

    def post(self, api_suffix, params):
        if isinstance(params, str):
            return requests.post(BASE_URL + api_suffix, data=params, headers=self.headers)
        else:
            qurey_str = self.form_encoding(api_suffix, params)
            return requests.post(BASE_URL + api_suffix, data=qurey_str, headers=self.headers)

    def nonce(self):
        chars = '0123456789abcdef'
        res = ''
        for _ in range(9):
            res += chars[math.floor(16 * random.random())]
        return res

    def xyz(self, api_suffix, params_list: list):
        md5_str = api_suffix + '?AppKey=joker&' + '&'.join(map(lambda x: '='.join(x), params_list))
        return hashlib.md5(md5_str.encode()).hexdigest()

    def form_encoding(self, api_suffix, params: dict):
        params_list = [entry for entry in params.items()]
        params_list.sort(key=lambda x: x[0])
        params_list.append(('nonce', self.nonce()))
        xyz_param = self.xyz(api_suffix, params_list)
        params_list.append(('xyz', xyz_param))
        return urlencode(params_list)

    @check_response_return_json
    def get_account_articles(self, uuid):
        params = {
            'uuid': uuid,
            'flag': 'true'
        }
        return self.post(GET_ACCOUNT_ARTICLES_URL, params)

    @check_response_return_json
    def search_weixin_official_account(self, keyword, filter='', order=''):
        """
        :param keyword: keyword
        :param filter: one or more in [nickname, tags, auth(认证主体), top500, ori(原创), server(服务号), certified(认证号)]
        separate by '|'. e.g. 'tags|top500'
        :param order: relation(综合-默认), NRI(新榜指数), collect(收藏人数)
        :return:
        """
        params = {
            'keyName': keyword,
            'order': order,
            'filter': filter,
            'hasDeal': 'false',
        }

        return self.post(SEARCH_OFFICIAL_WEIXIN_ACCOUNT, params)

    def __str__(self):
        return f'<{self.__class__.__name__}>'

    __repr__ = __str__


In [8]:
client = NewrankClient(token)

In [14]:
res = client.search_weixin_official_account('你好')

In [15]:
res.text

'{"success":true,"value":null}'

In [16]:
res.request.body

'filter=&hasDeal=false&keyName=%E4%BD%A0%E5%A5%BD&order=&nonce=dcaf74d8e&xyz=b4ac699e1cf6c360c1aeec9272a6315d'

In [37]:
res.request.headers

{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.84 Safari/537.36', 'Accept-Encoding': 'gzip, deflate, br', 'Accept': 'application/json, text/javascript, */*; q=0.01', 'Connection': 'keep-alive', 'Cookie': 'token=AA79F89BA13E514FE5BB69C90B7E8578; tt_token=true', 'X-Requested-With': 'XMLHttpRequest', 'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8', 'Origin': 'https://www.newrank.cn', 'Content-Length': '93'}

In [18]:
res.text

''

In [17]:
arts = client.get_account_articles('3A54574B487FD9E3A68C33301E167F1C')

In [18]:
arts

<Response [200]>

In [19]:
arts.json()

{'success': True, 'value': -999}

In [171]:
arts.request.body

'flag=true&uuid=3A54574B487FD9E3A68C33301E167F1C&nonce=754e3b8f1&xyz=3fd9304c2dae0ccc4c5eb45dc5b8ca84'

In [161]:
arts.request.headers

{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.84 Safari/537.36', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Cookie': 'token=AA79F89BA13E514FE5BB69C90B7E8578; tt_token=true', 'Content-Length': '100'}

In [163]:
arts.text

'{"timestamp":1513840252062,"status":400,"error":"Bad Request","exception":"org.springframework.web.bind.UnsatisfiedServletRequestParameterException","message":"Parameter conditions \\"uuid, flag\\" not met for actual request parameters: ","path":"/xdnphb/detail/getAccountArticle"}'